#LinkedIn Graphs and Network Analysis

In this lab we will do the following:

1. Get a LinkedIn API key <br/>
2. Use oauth2 to get an acceess token <br/>
3. First we are going to download our own LinkedIn data using the LinkedIn API. <br/>
4. Then we are exporting this data as a csv file to be able to import it into Gephi. <br/>
5. Before starting Gephi we will do some network analysis directly in python <br/>
6. We will analyze our data with the external tool Gephi <br/>

In [1]:
!pip install oauth2
!pip install unidecode

  Running setup.py bdist_wheel for oauth2
  Stored in directory: C:\Users\sanjiv\AppData\Local\pip\Cache\wheels\c9\88\f7\b4a9b64cc04af313cef10fcb14e6ada39499578ffd57a065a5
  Running setup.py bdist_wheel for httplib2
  Stored in directory: C:\Users\sanjiv\AppData\Local\pip\Cache\wheels\3b\9b\fd\f416b1427149ee285b78a0978ab7739a5153602137650760fb
Successfully built oauth2 httplib2


C:\Python27\lib\site-packages\pip\_vendor\requests\packages\urllib3\util\ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
You are using pip version 7.0.1, however version 7.1.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
C:\Python27\lib\site-packages\pip\_vendor\requests\packages\urllib3\util\ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning


  Running setup.py bdist_wheel for unidecode
  Stored in directory: C:\Users\sanjiv\AppData\Local\pip\Cache\wheels\a8\85\b6\71cbccded974d5a8e15cbe7d22066d87f34329d98c7ed5d786
Successfully built unidecode


You are using pip version 7.0.1, however version 7.1.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
C:\Python27\lib\site-packages\pip\_vendor\requests\packages\urllib3\util\ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning


In [2]:
%matplotlib inline
from collections import defaultdict
import json

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd

from matplotlib import rcParams
import matplotlib.cm as cm
import matplotlib as mpl

#colorbrewer2 Dark2 qualitative color table
dark2_colors = [(0.10588235294117647, 0.6196078431372549, 0.4666666666666667),
                (0.8509803921568627, 0.37254901960784315, 0.00784313725490196),
                (0.4588235294117647, 0.4392156862745098, 0.7019607843137254),
                (0.9058823529411765, 0.1607843137254902, 0.5411764705882353),
                (0.4, 0.6509803921568628, 0.11764705882352941),
                (0.9019607843137255, 0.6705882352941176, 0.00784313725490196),
                (0.6509803921568628, 0.4627450980392157, 0.11372549019607843)]

rcParams['figure.figsize'] = (10, 6)
rcParams['figure.dpi'] = 150
rcParams['axes.color_cycle'] = dark2_colors
rcParams['lines.linewidth'] = 2
rcParams['axes.facecolor'] = 'white'
rcParams['font.size'] = 14
rcParams['patch.edgecolor'] = 'white'
rcParams['patch.facecolor'] = dark2_colors[0]
rcParams['font.family'] = 'StixGeneral'


def remove_border(axes=None, top=False, right=False, left=True, bottom=True):
    """
    Minimize chartjunk by stripping out unnecesasry plot borders and axis ticks
    
    The top/right/left/bottom keywords toggle whether the corresponding plot border is drawn
    """
    ax = axes or plt.gca()
    ax.spines['top'].set_visible(top)
    ax.spines['right'].set_visible(right)
    ax.spines['left'].set_visible(left)
    ax.spines['bottom'].set_visible(bottom)
    
    #turn off all ticks
    ax.yaxis.set_ticks_position('none')
    ax.xaxis.set_ticks_position('none')
    
    #now re-enable visibles
    if top:
        ax.xaxis.tick_top()
    if bottom:
        ax.xaxis.tick_bottom()
    if left:
        ax.yaxis.tick_left()
    if right:
        ax.yaxis.tick_right()
        
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)

##1. Get API key

In [17]:
api_key = '77i6zo15tp47nk'
secret_key  = 'uDSORbQfGbfwr9E1'

##2. Get Access Token

In [18]:
# LinkedIn uses OAuth 2.0 for online verification.

import oauth2 as oauth
import urlparse
        
def request_token(consumer):
    client = oauth.Client(consumer)
    request_token_url = 'https://api.linkedin.com/uas/oauth/requestToken?scope=r_network'
    resp, content = client.request(request_token_url, "POST")
    if resp['status'] != '200':
        raise Exception("Invalid response %s." % resp['status'])
    request_token = dict(urlparse.parse_qsl(content))
    return request_token

In [19]:
def authorize(request_token):
    authorize_url ='https://api.linkedin.com/uas/oauth/authorize'
    print "Go to the following link in your browser:"
    print "%s?oauth_token=%s" % (authorize_url, request_token['oauth_token'])
    print
    accepted = 'n'
    while accepted.lower() == 'n':
        accepted = raw_input('Have you authorized me? (y/n) ')
    oauth_verifier = raw_input('What is the PIN? ')
    return oauth_verifier

In [20]:
def access(consumer, request_token, oauth_verifier):
    access_token_url = 'https://api.linkedin.com/uas/oauth/accessToken'
    token = oauth.Token(request_token['oauth_token'], request_token['oauth_token_secret'])
    token.set_verifier(oauth_verifier)
    client = oauth.Client(consumer, token)
    resp, content = client.request(access_token_url, "POST")
    access_token = dict(urlparse.parse_qsl(content))
    return access_token

In [21]:
consumer = oauth.Consumer(api_key, secret_key)

r_token = request_token(consumer)
print "Request Token: oauth_token: %s, oauth_token_secret: %s" % (r_token['oauth_token'], r_token['oauth_token_secret'])

oauth_verifier = authorize(r_token)

a_token = access(consumer, r_token, oauth_verifier)
print a_token
print "Access Token: oauth_token = %s, oauth_token_secret = %s" % (a_token['oauth_token'], a_token['oauth_token_secret'])
print "You may now access protected resources using the access tokens above."

Exception: Invalid response 403.

In [22]:
client = oauth.Client(consumer)

In [26]:
request_token_url = 'https://api.linkedin.com/uas/oauth/requestToken?scope=r_network'
resp, content = client.request(request_token_url, "POST")

In [27]:
resp

{'-content-encoding': 'gzip',
 'connection': 'keep-alive',
 'content-length': '54',
 'content-type': 'application/x-www-form-urlencoded;charset=UTF-8',
 'date': 'Sat, 25 Jul 2015 05:32:47 GMT',
 'server': 'Apache-Coyote/1.1',
 'set-cookie': 'lidc="b=VB19:g=289:u=1:i=1437802368:t=1437888768:s=AQFhSMoaLOwv5qJ0ryXm-49dgtKaU2TQ"; Expires=Sun, 26 Jul 2015 05:32:48 GMT; domain=.linkedin.com; Path=/',
 'status': '403',
 'transfer-encoding': 'chunked',
 'vary': 'Accept-Encoding',
 'www-authenticate': 'OAuth realm="https%3A%2F%2Fapi.linkedin.com", oauth_problem="Scope%20NOT_AUTHORIZED%20%3A%20r_network"',
 'x-fs-uuid': '9ceb71d4a219f41380b78a6abb2a0000',
 'x-li-fabric': 'prod-lva1',
 'x-li-pop': 'prod-lva1',
 'x-li-uuid': 'nOtx1KIZ9BOAt4pquyoAAA=='}

In [28]:
content

'oauth_problem=Scope%20NOT_AUTHORIZED%20%3A%20r_network'